# 図表を作成する
このノートブックではコードセルを使用して図表を作成することが可能です。<br>
ノートブックは図表ごとに作成し、図表と元データの関連性がわかるようにノートブックに明記した上で図表と共に管理してください。<br>
<span style="color:red">※このノートブックを編集した際は必ず保存してから終了してください。</span>

## 入力データを表示する
グラフの作成に用いるデータを読み込みます。<br>
「論拠データを表示する」セルを実行して表示した論拠データのフォルダから読み込ませたいデータを選択し、「ファイルを読み込む」セルにファイル名を書き込んでください。<br>
必要なファイル名を書き込んだ後、セルを実行することでファイルを読み込むことが可能です。<br>
ファイルは複数選択することも可能です。<br>
<br>
グラフの作成には論拠データとして保存したものを使用してください。<br>
論拠データとして保存していないデータを使用したい場合は論拠データに追加してください。<br>
現在、読み込み可能なデータの形式はcsv、text(csv形式で記述）、excelのみとなっています。<br>
それ以外の形式のファイルを読み込む場合はコードセルを修正してください。<br>

In [ ]:
# 論拠データを表示する
import os

from IPython.core.display import Javascript
from IPython.display import display
import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button
from library.utils import file


def access_argument_data(working_path):
    """論拠データにアクセスするためのメソッドです。

    Args:
        working_path (_type_): 実行Notebookファイルパス

    """
    folder_name = 'argument_data'
    button_width = 500

    # /dataからargument_dataまでのパス
    home = os.environ['HOME']
    data_dir = file.relative_path(working_path, home)
    data_dir = os.path.dirname(data_dir)
    argument_dir = os.path.join(os.path.dirname(data_dir), folder_name)
    # 現在地からhomeまでのパス
    script_dir = os.path.dirname(working_path)
    script_dir = file.relative_path(home, script_dir)
    # ディレクトリを表示するのでtreeにする
    url = os.path.join(script_dir, '../tree/', argument_dir)
    # ボタンの作成
    obj = create_button(
        url=url,
        target='_blank',
        msg=msg_config.get('task', 'access_data_dir'),
        button_width=f'{button_width}px'
    )
    # 表示
    pn.extension()
    display(pn.pane.HTML(obj))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_argument_data(os.path.abspath('__file__'))

In [ ]:
# ファイルを読み込む
import os

from IPython.display import display, Javascript
import matplotlib.pyplot as plt
import panel as pn
import pandas as pd

from library.utils.config import message as msg_config
from library.utils.widgets import MessageBox


load_files = [""] # ここに読み込むファイル名を書く(例：load_files =["sample.text", "sample2.csv"])


output_data = {}

def load_file(working_path):
    """指定されたファイルを読み込んで表示するメソッドです。

    Args:
        working_path (_type_): 実行Notebookファイルパス

    """
    folder_name = 'argument_data'

    # 読み込むファイルが見つからない場合のメッセージボックス
    load_not_found_message = MessageBox()
    load_not_found_message.width = 900
    # 読み込むファイルの形式が間違っていた場合のメッセージボックス
    load_invalid_message = MessageBox()
    load_invalid_message.width = 900
    # ファイルが読み込めない場合のメッセージボックス
    not_load_message = MessageBox()
    not_load_message.width = 900

    pn.extension()

    # ファイル名が指定されていない
    if not load_files:
        load_not_found_message.update_warning(
            msg_config.get('figure_template', 'file_not_specify'))
        display(load_not_found_message)
        return

    # データの保存場所までの絶対パス
    data_dir = os.path.dirname(working_path)
    argument_dir = os.path.join(os.path.dirname(data_dir), folder_name)

    argument_files = {}
    not_exist_files = []
    invalid_files = []
    # argument_data配下の全てのファイルを走査
    for root, dirs, files in os.walk(argument_dir):
        for file in files:
            argument_files[file] = os.path.join(root, file)

    # 指定したファイルが読み込めるかの判定
    for file in load_files:
        try:
            if file in argument_files:
                extension = os.path.splitext(file)[1]
                if extension == ".csv" or extension == ".txt": # 既存の形式以外のファイルを読み込みたい場合はここに追加する
                    output_data[file] = pd.read_csv(argument_files[file])
                    display(output_data[file])
                elif extension == "xlsx":
                    output_data[file] = pd.read_excel(argument_files[file]) # 特定のシートを読み込みたい場合はsheet_name = "")を引数に加える
                    display(output_data[file])

                else:
                    invalid_files.append(file)
            else:
                not_exist_files.append(file)

        except Exception as e:
            not_load_message.update_error(msg_config.get(
            'figure_template', 'file_not_load') + " " + file + " " + e)
            display(not_load_message)
            continue

    # ファイルが存在しない場合のメッセージ
    if not_exist_files:
        load_not_found_message.update_warning(msg_config.get(
            'figure_template', 'file_not_found') + " " + ", ".join(not_exist_files))
        display(load_not_found_message)
    # ファイルが指定された形式でない場合のメッセージ
    if invalid_files:
        load_invalid_message.update_warning(msg_config.get(
            'figure_template', 'file_invalid') + " " + ", ".join(invalid_files))
        display(load_invalid_message)

    display(Javascript('IPython.notebook.save_checkpoint();'))

load_file(os.path.abspath('__file__'))

## 図表を作成する
論拠データを使用して図表の作成を行ってください。

### 1. 表を作成する
このセルでは、読み込んだデータファイルが指定されなかった場合にサンプルの表が生成されるようになっています。<br>
表を作成する際は使用するファイル名を書き込み、コードを書き換えて使用してください。<br>
セルを実行することで作成した表を出力することができます。<br>

表の作成にはpandasライブラリを使用しています。<br>
ライブラリの使い方を知りたい場合は[こちら](https://pandas.pydata.org/)をクリックしてください。<br>

また、サンプルのコードでは以下のようなcsvファイルを想定しています。<br>
![図表用のサンプルscv](../../../../data_governance/images/figure_template_table.png)

In [ ]:
# コードセルで作成する
from IPython.display import display
import pandas as pd


file_name = ""  # 読み込んだファイルの内、使用するファイルの名前を入力してください（例：file_name = "sample.csv")
                # 「ファイルを読み込む」セルで読み込んだファイルを指定してください


# file_nameを入力していない場合はサンプルのデータを出力する
if not file_name:
    sample_data = {
        'Category': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 15, 7, 20, 5],
        'Value2': [5, 7, 3, 10, 2]
    }
    sample_df = pd.DataFrame(sample_data)
    output_data = sample_df

else:
    output_data = output_data[file_name]
    display(output_data)


# これより下に出力したい表の設定を記載する

output_data = output_data[['Category','Value2']] # 例：'Category'と'Value2の列のみを出力





# 表を出力
display(output_data)

#### 作成した表を保存する<br>
作成した表の保存を行います。<br>
コードセルに保存するファイル名を書き込み、実行してください。<br>
作成した表はノートブックと同じフォルダに保存されます。<br>
また、表は画像ファイル(png)として保存されます。<br>
異なる形式で出力したい場合はセルのコードを書き換えて下さい。


In [ ]:
# 表を保存する
import matplotlib.pyplot as plt


table_name = "sample.png" # 保存する際の名前を入力してください(例："sample.png")


# matplotlibのFigureとAxesを設定
fig, ax = plt.subplots(figsize=(8, 4))  # 表のサイズを指定

# 表をプロット（データフレームをmatplotlibのtable形式で表示）
ax.axis('tight')
ax.axis('off')  # 軸は非表示にする

# データフレームをmatplotlibのtableに変換して表示
table = ax.table(cellText=output_data.values, colLabels=output_data.columns, loc='center')

# 画像として保存
plt.savefig(table_name, format='png', bbox_inches='tight', dpi=300)

### 2. グラフを作成する
このセルでは、読み込んだデータファイルが指定されなかった場合にサンプルの棒グラフが生成されるようになっています。<br>
グラフを作成する際は使用するファイル名を書き込み、コードを書き換えて使用してください。<br>
セルを実行することで作成したグラフを出力することができます。<br>

グラフの作成にはmatplotlibライブラリを使用しています。<br>
ライブラリの使い方を知りたい場合は[こちら]( https://matplotlib.org/)をクリックしてください。<br>

また、サンプルのコードでは以下のようなcsvファイルを想定しています。<br>
![グラフ用のサンプルcsv](../../../../data_governance/images/figure_template_graph.png)

In [ ]:
# コードセルで作成する
import matplotlib.pyplot as plt
import pandas as pd


file_name = ""  # 読み込んだファイルの内、使用するファイルの名前を入力してください（例：file_name = "sample.csv")
                # 「ファイルを読み込む」セルで読み込んだファイルを指定してください


# file_nameを入力していない場合はサンプルのデータを出力する
if not file_name:
    sample_data = {
        'Category': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 15, 7, 20, 5],
        'Value2': [5, 7, 3, 10, 2]
    }
    sample_df = pd.DataFrame(sample_data)
    output_data = sample_df

else:
    output_data = output_data[file_name]
    display(output_data)


# これより下にグラフの設定を記載する

# 'Category'をx軸、'Value'をy軸にして棒グラフを作成
ax = output_data.plot(kind='bar', x='Category', y='Value', legend=False)

# タイトルと軸ラベルを追加
plt.title('Value by Category')
plt.xlabel('Category')
plt.ylabel('Value')





#### 作成したグラフを保存する<br>
作成したグラフの保存を行います。<br>
コードセルに保存するファイル名を書き込み、実行してください。<br>
作成したグラフはノートブックと同じフォルダに保存されます。<br>
また、グラフは画像ファイル(png)として保存されます。<br>
異なる形式で出力したい場合はセルのコードを書き換えて下さい。


In [ ]:
# グラフを保存する

graph_name = "sample_graph.png" # 保存する際の名前を入力してください(例："sample_graph.png")


# pngとして出力
ax.get_figure().savefig(graph_name, format='png', bbox_inches='tight', dpi=300)
display(ax.get_figure())